# Figure 6: Regionalization differences (map)

In [ ]:
import os 
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import shapely.geometry

os.chdir('/home/rooda/OneDrive/Projects/DeepHydro/')
path_pmet = "/home/rooda/OneDrive/PatagoniaMet/"
path_geo = "/home/rooda/Datasets/GIS/"

## Data (historical)

In [ ]:
q_data = xr.open_dataset("results/zenodo/Q_historical.nc")
q_data = q_data.resample(date = "YS").sum().mean(dim = "date")
q_data = q_data.to_dataframe().reset_index()
q_data = q_data.pivot(index="basin_id", columns="model", values="Q")

In [ ]:
pmet_shape = gpd.read_file("data/GIS/Basins_Patagonia_all_data.gpkg") # Problem in Puelo 
pmet_shape = pmet_shape.set_index("gauge_id")
pmet_shape["total_area"] = pmet_shape.total_area*1e6 # in m2 
pmet_shape   = pd.concat([pmet_shape, q_data], axis = 1)
pmet_shape["Model_STD"]   = pmet_shape[["LSTM_OGGM_on", "LSTM_OGGM_off", "TUWmodel", "GR4J"]].std(axis = 1)
pmet_shape["Model_MEAN"]  = pmet_shape[["LSTM_OGGM_on", "LSTM_OGGM_off", "TUWmodel", "GR4J"]].mean(axis = 1)

pmet_shape_subset = pmet_shape[((pmet_shape.LSTM_OGGM_on-pmet_shape.Model_MEAN)/pmet_shape.Model_STD).abs() > 1]

## Plot elements

In [ ]:
# basemap for background
geo_map = gpd.read_file(path_geo + "south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = shapely.geometry.Polygon([(-76, -54.99), (-76, -40.51), (-68.01, -40.51), (-68.01, -54.99), (-76, -54.99)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)

geo_map = geo_map.clip(poly_gdf)

In [ ]:
pmet_shape_main = pmet_shape[pmet_shape["total_area"] > 5000*1e6]
pmet_shape_main["basin_name"] = ["Santa Cruz                          ", "Puelo", "Yelcho", "Grey","Palena", "Pascua", "Aysen", "Baker", "Cisnes"]

## Plot

In [ ]:
fig = make_subplots(rows=1, cols=2, vertical_spacing = 0.01, horizontal_spacing = 0.01,
                    specs=[[{"type": "scattergeo"}, {"type": "scattergeo"}]])

# colors
cl = px.colors.qualitative.D3
cs = px.colors.colorbrewer.GnBu
colorsc_a = [cs[1], cs[4], cs[8]]
colorsc_b = [(0,"#fe7e0d"),(0.5, "rgba(255,233,186,0.7)"),(1, "#1d78b4")]

dtick = 2
x = list(range(-74, 0 + dtick, dtick))
y = list(range(-56, 0 + dtick, dtick))
xpos = -75.9
ypos = -56

## Basemap -----------------------------------------------------------------------------------------------
for x_plot in range(0,2):
        fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                                    colorscale = ["#EAEAF2", "#EAEAF2"],  showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=x_plot+1)
    
        fig.add_trace(go.Scattergeo(lon = [lon + 0.25 for lon in x[1:-1]] + [xpos + 0.25] * (len(y) - 2),
                                lat = [ypos + 0.1] * (len(x) - 2) + [lat + 0.1 for lat in y[1:-1]],
                                showlegend = False,
                                text = x[1:-1] + y[1:-1], textfont=dict(size=11, color = "rgba(0,0,0,0.25)"),
                                mode = "text"), row=1, col=x_plot+1)

fig.add_scattergeo(geojson = eval(pmet_shape_main['geometry'].to_json()), locations = pmet_shape_main.index, text = pmet_shape_main['basin_name'], mode = 'text', 
                           showlegend = False, textfont=dict(size=11, color = "rgba(0,0,0,0.3)"),row=1, col=1)

fig.add_trace(go.Choropleth(geojson = eval(pmet_shape['geometry'].to_json()),  locations = pmet_shape.index, z = pmet_shape['LSTM_OGGM_on'], 
                            colorscale = colorsc_a, marker_line_color ='white', marker_line_width=0.2, 
                            zmin = 0, zmax = 7000, colorbar=dict(len=0.4, x=0.36, y= 0.80, title='Runoff <br>(mm yr<sup>-1</sup>)', thickness=15, tickwidth=1)), row=1, col=1)

fig.add_trace(go.Choropleth(geojson = eval(pmet_shape['geometry'].to_json()),  locations = pmet_shape.index, z = pmet_shape.Model_STD/pmet_shape.Model_MEAN, 
                            colorscale = colorsc_a, marker_line_color ='white', marker_line_width=0.2, 
                            zmin = 0, zmax = 0.6, colorbar=dict(len=0.4, x=0.87, y= 0.80, title='CV (-)', thickness=15, tickwidth=1)), row=1, col=2)

fig.add_trace(go.Choropleth(geojson = eval(pmet_shape['geometry'].to_json()), locations = pmet_shape_subset.index,  z = pmet_shape_subset.Model_STD, 
                            colorscale = ["rgba(0,0,0,0)", "rgba(0,0,0,0)"], marker_line_color ='black', marker_opacity = 0.5,
                            showscale= False, marker_line_width=0.3), row=1, col=2)

## Annotations -----------------------------------------------------------------------------------------------
fig.add_annotation(text="a) Runoff (LSTM + OGGM)",  x=0.01, y=0.001,  xref = "paper", yref = "paper", showarrow=False)
fig.add_annotation(text="b) Multi-model variability", x=0.62, y=0.0001,  xref = "paper", yref = "paper", showarrow=False)

## Layout -------------------------------------------------------------------------------------------------------------------
fig.update_xaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", griddash = "dot", mirror=True)
fig.update_yaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", griddash = "dot", mirror=True)

fig.update_geos(showframe = True, framewidth = 1, framecolor = "rgba(0,0,0,0.5)", 
                lonaxis_range=[-76, -68], lataxis_range=[-56, -40.5], 
                bgcolor = "rgb(255,255,255)", 
                showland = False, showcoastlines = False, showlakes = False, 
                lataxis_showgrid=True, lonaxis_showgrid=True, 
                lonaxis_dtick = dtick, lataxis_dtick = dtick, 
                lonaxis_gridcolor = "rgba(0,0,0,0.1)", lataxis_gridcolor = "rgba(0,0,0,0.1)", 
                lonaxis_griddash = "dot", lataxis_griddash = "dot")

fig.update_layout(autosize = False, width = 650, height = 600, template = "seaborn", margin = dict(l=5, r=5, b=5, t=5))
fig.write_image("reports/figures/Figure6_Regionalization_diff_map.png", scale = 4)
#fig.show()

# Text

In [ ]:
data_runoff = xr.open_dataset("results/runoff/total_runoff_historical_LSTM_OGGM_on_all.nc").Q ## mm d-1
data_runoff = data_runoff.resample(date = "YS").sum()
data_runoff = data_runoff.to_pandas().transpose()
data_runoff = data_runoff * pmet_shape.total_area/(365*1e3*86400)
"The total runoff flux for the study area is {:.0f} +- {:.0f} m3/s".format(data_runoff.sum(axis = 1).mean(), data_runoff.sum(axis = 1).std())

In [ ]:
data_glacier_runoff = pd.read_csv("results/runoff/glacier_runoff_historical_all.csv", index_col = 0, parse_dates= ["time"])
data_glacier_runoff = data_glacier_runoff[data_glacier_runoff.index.year >= 2000]
data_glacier_runoff = data_glacier_runoff.resample("YS").mean().sum(axis = 1)

data_glacier_melt = pd.read_csv("results/runoff/glacier_melt_historical_all.csv", index_col = 0, parse_dates= ["time"])
data_glacier_melt = data_glacier_melt[data_glacier_melt.index.year >= 2000]
data_glacier_melt = data_glacier_melt.resample("YS").mean().sum(axis = 1)

"Glacier runoff and melt accounted for {:.0f} ± {:.0f}% and {:.0f}% ± {:.0f}% of this total".format(data_glacier_runoff.mean()*100 / data_runoff.sum(axis = 1).mean(), 
                                                                                                    data_glacier_runoff.std()*100  / data_runoff.sum(axis = 1).std(),
                                                                                                    data_glacier_melt.mean()*100   / data_runoff.sum(axis = 1).mean(),
                                                                                                    data_glacier_melt.std()*100    / data_runoff.sum(axis = 1).std()  )

In [ ]:
"When analysed by latitude, the northern, central, and southern zones contributed {:.0f}%, {:.0f}%, and {:.0f}% of the total discharge".format(
    data_runoff.mean()[pmet_shape.lat > -46].sum() * 100 / data_runoff.mean().sum(),
    data_runoff.mean()[(pmet_shape.lat < -46) & (pmet_shape.lat > -51)].sum() * 100 / data_runoff.mean().sum(),
    data_runoff.mean()[pmet_shape.lat < -51].sum() * 100 / data_runoff.mean().sum())

In [ ]:
pmet_shape["LSTM_OGGM_on_flux"] = pmet_shape.LSTM_OGGM_on * pmet_shape.total_area/(365*1e3*86400)
pmet_shape.LSTM_OGGM_on_flux.groupby(pmet_shape.lat.round(0)).sum().sort_index()


In [ ]:
# consistency between LSTM OGGM_on and alternatives
"{:.1f}% of the total study area showed agreement in the annual runoff ".format(pmet_shape_subset.total_area.sum()*100/pmet_shape.total_area.sum())

In [ ]:
"{:.1f}% of the total study area showed a CV lower than 0.2".format(
    pmet_shape[(pmet_shape.Model_STD/pmet_shape.Model_MEAN) > 0.2].total_area.sum()*100 / pmet_shape.total_area.sum())